# Подключение нужных библиотек

Первым шагом является импорт следующего списка библиотек:

In [2]:
import speech_recognition as sr
import urllib
import os
import json
import io
import pandas as pd
from pydub import AudioSegment
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score 
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec
#for BERT neural model
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ilars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ilars\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ilars\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ilars\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Загрузка нужных данных из локальных носителей

In [ ]:
AudioSegment.ffmpeg = "c:/users/ilars/anaconda3/lib/site-packages/ffmpeg"


#Преобразовываем аудиофайлы положительных звонков в новый формат и переводим их в текстовый формат для дальнейшей обработки
directory = "C:\Speach\succ"

for filename in os.listdir("C:/Speach/succ"):
    filepath = os.path.join(directory, filename)
    sound = AudioSegment.from_mp3(filepath)
    sound.export("C:/Speach/new_succ/"+filename[:-3]+"wav", format="wav", parameters=["-ac", "2", "-ar", "8000"])

directory = "C:/Speach/new_succ/"

texts = []

for filename in os.listdir("C:/Speach/new_succ"):
    r = sr.Recognizer()
    filepath = os.path.join(directory, filename)
    file = sr.AudioFile(filepath)
    with file as source:
        r.adjust_for_ambient_noise(source)
        try:
            audio = r.record(source)
            result = r.recognize_google(audio,language="ru")
        except:
            continue
    texts.append({"text":result, "y":1})
    continue
    
    
#Преобразовываем аудиофайлы отрицательных звонков в новый формат и переводим их в текстовый формат для дальнейшей обработки
directory = "C:/Speach/non_succ/"

for filename in os.listdir("C:/Speach/non_succ"):
    filepath = os.path.join(directory, filename)
    sound = AudioSegment.from_mp3(filepath)
    sound.export("C:/Speach/new_non/"+filename[:-3]+"wav", format="wav", parameters=["-ac", "2", "-ar", "8000"])

directory = "C:/Speach/new_non/"

for filename in os.listdir("C:/Speach/new_non"):
    r = sr.Recognizer()
    filepath = os.path.join(directory, filename)
    file = sr.AudioFile(filepath)
    with file as source:
        r.adjust_for_ambient_noise(source)
        try:
            audio = r.record(source)
            result = r.recognize_google(audio,language="ru")
        except:
            continue
    texts.append({"text":result, "y":0})
    continue

Далее стоит записать полученные тексты в json-формат и получить удобный для обработки датасет

In [ ]:
io.open("speach_data.json", "w", encoding="utf-8").write(json.dumps(texts, ensure_ascii=False))

In [3]:
#Полученный датасет в формате pd.DataFrame
df_train = pd.read_json('speach_data.json')
df_train.head()

,text,y
0,заполненная заявка на Здравствуйте Вас приветс...,1
1,Здравствуйте Вас приветствует официальный диле...,1
2,Здравствуйте Вас приветствует Авангард официал...,1
3,Здравствуйте Вас приветствует официальный диле...,1
4,Здравствуйте Вас приветствует Авангард официал...,1


# Предобработка данных

Прежде чем мы перейдем к построению модели, нам необходимо предварительно обработать наш набор данных, удалив знаки препинания и специальные символы, очистив тексты, удалив стоп-слова и применив лемматизацию.


Простые процессы очистки текста: некоторые из распространенных процессов очистки текста включают:

-Удаление знаков препинания, специальных символов, URL-адресов и хэштегов

-Удаление начальных, завершающих и лишних пробелов / табуляций

-Исправлены опечатки, сленги, аббревиатуры написаны в их длинных формах

-Удаление стоп-слов: мы можем удалить список общих стоп-слов из английского словаря с помощью nltk. Несколько таких слов - "я’, ’ты’, ’а", "тот", "он", "который’ и т.д.

-Стемминг: относится к процессу вырезания конца или начала слов с намерением удаления аффиксов (префикс / суффикс)

-Лемматизация: это процесс приведения слова к его базовой форме

In [4]:
# преобразовать в нижний регистр, зачеркнуть и удалить знаки препинания
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# Удаление стоп-слов
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
# Лемматизация

wl = WordNetLemmatizer()
 
# Это вспомогательная функция для сопоставления тегов положения NTLK
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Токенизация
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Получение тегов положения
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Сопоставление тегов позиции и лемматизация слов/токенов
    return " ".join(a)

В результате получаются тексты, готовые для обработки моделью

In [5]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

df_train['clean_text'] = df_train['text'].apply(lambda x: finalpreprocess(x))
df_train.head()

,text,y,clean_text
0,заполненная заявка на Здравствуйте Вас приветс...,1,заполненная заявка на здравствуйте вас приветс...
1,Здравствуйте Вас приветствует официальный диле...,1,здравствуйте вас приветствует официальный диле...
2,Здравствуйте Вас приветствует Авангард официал...,1,здравствуйте вас приветствует авангард официал...
3,Здравствуйте Вас приветствует официальный диле...,1,здравствуйте вас приветствует официальный диле...
4,Здравствуйте Вас приветствует Авангард официал...,1,здравствуйте вас приветствует авангард официал...


# Построение модели и обучение

Для упрощения работы используется кастомизированный класс Dataset:

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=512):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }

Класс CustomDataset необходим для использования с библиотекой transformers. Наследуется от класса Dataset. В нем определяются 3 обязательные функции: init, len, getitem. основное предназначение - возвращает токенизированные данные в нужном формате.

In [ ]:
from transformers import BertTokenizer
tokenizer_path = 'cointegrated/rubert-tiny'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

Метод len возвращает длину нашего датасета. Метод getitem возвращает словарь, который состоит из самого исходного текста, списка токенов, маски внимания, а также метки класса. Отдельно хочется остановить на настройках токенизатора с помощью метода .encode_plus(). В этом методе мы указываем токенизатору, что исходный текст нужно обрамлять служебными токенами add_special_tokens=True, а также дополнять полученные векторы до максимально длины padding='max_len'.

## Модель

Задаем класс BertClassifier для работы с ним


### Инициализация:

- Скачиваются модель и токенизатор из репозитория huggingface
- Определяется наличие целевого устройства для вычислений
- Определяется размерность ембеддингов
- Задается количество классов
- Задается количество эпох для обучения

### Preparation
Для обучения BERT нужно инициализировать несколько вспомогательных элементов:

   - DataLoader: нужен для создания батчей;
   - Optimizer: оптимизатор градиентного спуска;
   - Scheduler: планировщик, нужен для настройки параметров оптимизатора;
   - Loss: функция потерь, считаем по ней ошибку модели.

### Train
- Обучение для одной эпохи описано в методе fit.
    - Данные в цикле батчами генерируются с помощью DataLoader;
    - Батч подается в модель;
    - На выходе получаем распределение вероятности по классам и значение ошибки;
    - Делаем шаг на всех вспомогательных функциях:
        - loss.backward: обратное распространение ошибки;
        - clip_grad_norm: обрезаем градиенты для предотвращения "взрыва" градиентов;
        - optimizer.step: шаг оптимизатора;
        - scheduler.step: шаг планировщика;
        - optimizer.zero_grad: обнуляем градиенты.
- Проверку на валидационной выборке проводим с помощью метода eval. При этом используем метод torch.no_grad для предотвращения обучения на валидационной выборке.
- Для обучения на нескольких эпохах используется метод train, в котором последовательно вызываются методы fit и eval.

### Inference
Для предсказания класса для нового текста используется метод predict, который имеет смысл вызывать только после обучения модели.
Метод работает следующим образом:

- Токенизируется входной текст;
- Токенизированный текст подается в модель;
- На выходе получаем вероятности классов;
- Возвращаем метку наиболее вероятного класса.

In [ ]:
class BertClassifier:

    def __init__(self, model_path, tokenizer_path, n_classes=2, epochs=1, model_save_path='bert.pt'):
        self.model = BertForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_save_path=model_save_path
        self.max_len = 512
        self.epochs = epochs
        self.out_features = self.model.bert.encoder.layer[1].output.dense.out_features
        self.model.classifier = torch.nn.Linear(self.out_features, n_classes)
        self.model.to(self.device)
    
    def preparation(self, X_train, y_train, X_valid, y_valid):
        # create datasets
        self.train_set = CustomDataset(X_train, y_train, self.tokenizer)
        self.valid_set = CustomDataset(X_valid, y_valid, self.tokenizer)

        # create data loaders
        self.train_loader = DataLoader(self.train_set, batch_size=2, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=2, shuffle=True)

        # helpers initialization
        self.optimizer = AdamW(self.model.parameters(), lr=2e-5, correct_bias=False)
        self.scheduler = get_linear_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=0,
                num_training_steps=len(self.train_loader) * self.epochs
            )
        self.loss_fn = torch.nn.CrossEntropyLoss().to(self.device)
            
    def fit(self):
        self.model = self.model.train()
        losses = []
        correct_predictions = 0

        for data in self.train_loader:
            input_ids = data["input_ids"].to(self.device)
            attention_mask = data["attention_mask"].to(self.device)
            targets = data["targets"].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            preds = torch.argmax(outputs.logits, dim=1)
            loss = self.loss_fn(outputs.logits, targets)

            correct_predictions += torch.sum(preds == targets)

            losses.append(loss.item())
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()

        train_acc = correct_predictions.double() / len(self.train_set)
        train_loss = np.mean(losses)
        return train_acc, train_loss
    
    def eval(self):
        self.model = self.model.eval()
        losses = []
        correct_predictions = 0

        with torch.no_grad():
            for data in self.valid_loader:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss_fn(outputs.logits, targets)
                correct_predictions += torch.sum(preds == targets)
                losses.append(loss.item())
        
        val_acc = correct_predictions.double() / len(self.valid_set)
        val_loss = np.mean(losses)
        return val_acc, val_loss
    
    def train(self):
        best_accuracy = 0
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_acc, train_loss = self.fit()
            print(f'Train loss {train_loss} accuracy {train_acc}')

            val_acc, val_loss = self.eval()
            print(f'Val loss {val_loss} accuracy {val_acc}')
            print('-' * 10)

            if val_acc > best_accuracy:
                torch.save(self.model, self.model_save_path)
                best_accuracy = val_acc

        self.model = torch.load(self.model_save_path)
    
    def predict(self, text):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }
        
        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)
        
        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )
        
        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction

Разделим существующую обучающую выборку на тренировочный и тестовый для проверки результата

In [ ]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df_train["clean_text"],df_train["y"],test_size=0.45,shuffle=True)

Инициализируем модель

In [ ]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=20,
        model_save_path='bert.pt'
)

Подготовим данные

In [ ]:
classifier.preparation(
        X_train=list(X_train),
        y_train=list(y_train),
        X_valid=list(X_test),
        y_valid=list(y_test)
    )

Обучение модели

In [ ]:
classifier.train()

# Выгрузка модели
Данная модель сохраняется в файл 'bert.pt'и в дальнейшем может интегрироваться в веб-приложения, программные обеспечения и т.д.
Пример выгрузки модели написан ниже с помощью функции модуля torch.

In [ ]:
model = BertClassifier(model_path='cointegrated/rubert-tiny',tokenizer_path='cointegrated/rubert-tiny')
model.model = torch.load('bert.pt')